In [1]:
import os
import pandas as pd
from langdetect import detect
import re
import nltk
import string
from nltk.corpus import stopwords
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
from collections import Counter
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from nltk.stem import WordNetLemmatizer, PorterStemmer
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
def concatenate_csv_files_in_folder(folder_path):
    
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
    
    dfs = []
    
    for csv_file in csv_files:
        try:
            file_path = os.path.join(folder_path, csv_file)
            df = pd.read_csv(file_path, sep=',', encoding='latin-1')
            dfs.append(df)
        except Exception as e:
            print(f"Error reading {csv_file}: {str(e)}")
            
    concatenated_df = pd.concat(dfs, ignore_index=True)
    
    return concatenated_df

In [3]:
folder_path = './'
result_df = concatenate_csv_files_in_folder(folder_path)

In [4]:
result_df

,title,body
0,fit error help,model = Sequential()\r\n\r\nmodel.add(Conv2D(3...
1,éªè¯ç éå,NaN
2,add a font file and fix the writing style,NaN
3,4-6ä¸ªå­ç¬¦è½ç¨ä¸ä¸ªæ¨¡åå®ç°åï¼,ä½ å¥½ï¼ä½ çæ¨¡åææåå¤å¶äºï¼4ä¸ª...
4,write labels err,writer.writerows(labels)\r\nTypeError: a b...
...,...,...
121412,Fix renew_ch,NaN
121413,convert Tensorflow weights to Pytorch weights,I have converted Tensorflow weights to Pytorch...
121414,In_channels in last conv should be renew_ch(320)?,https://github.com/zsef123/EfficientNets-PyTor...
121415,Why you don't use any skip connection?,https://github.com/zsef123/EfficientNets-PyTor...


In [7]:
def filter_english_french_rows(df, min_text_length=10):
    
    filtered_rows = []
    
    for index, row in df.iterrows():
        title = row['title']
        body = row['body']
        if pd.notna(str(title)) and pd.notna(str(body)):
            try:
                title_lang = detect(str(title))
                body_lang = detect(str(body))
                
                # Check if both title and body are in English or French
                if (title_lang in ['en', 'fr'] and len(str(title)) >= min_text_length) and \
                   (body_lang in ['en', 'fr'] and len(str(body)) >= min_text_length):
                    filtered_rows.append(row)
            except:
                print('An exception occurred')
                pass 
            
    filtered_df = pd.DataFrame(filtered_rows)
    filtered_df.reset_index(drop=True, inplace=True)
    return filtered_df

In [8]:
filtered_df = filter_english_french_rows(result_df)

An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An excepti

An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An excepti

In [9]:
filtered_df

,title,body
0,numpy.core._internal.AxisError: axis 1 is out ...,è£ å¥½ä¾èµï¼çæå¾çåï¼ç´æ¥è¿è¡ï...
1,Only extract one word from gumbel softmax,In the code https://github.com/yala/text_nn/bl...
2,Gumbel softmax instead of Reinforce,"Hi,\r\n\r\nThank you for your code ! I have se..."
3,Is log_softmax function missing in the Line 39...,I think log_softmax function should be added i...
4,Can this be applied to a regression problem?,Can this be applied to a regression problem?
...,...,...
44871,the meaning of depth div,Hi\r\n\r\nwhat is the meaning of `depth_div` u...
44872,accuracy for the code,"Hi,\r\n\r\nThanks for your code. Did you do so..."
44873,convert Tensorflow weights to Pytorch weights,I have converted Tensorflow weights to Pytorch...
44874,Why you don't use any skip connection?,https://github.com/zsef123/EfficientNets-PyTor...


In [10]:
filtered_df.to_csv('Issue_collected.csv')

In [1]:
# Download the stopwords corpus
nltk.download('stopwords')
nltk.download('wordnet')

NameError: name 'nltk' is not defined

In [ ]:
def preprocess_text(df):
    remove_infrequent=True
    threshold = 2
    infrequent_threshold=2
    # drop duplicates
#     df = df.drop_duplicates()
    # drop NAN
    df = df.dropna()
    # Combine title and body into a single document
    df.loc[:, 'document'] = df['title'] + ' ' + df['body']

    # Convert text to lowercase
    df.loc[:, 'document'] = df.loc[:, 'document'].str.lower()

    # Remove special characters and punctuation
    df.loc[:, 'document'] = df.loc[:, 'document'].str.replace(r'[^a-zA-Z\s]', '')

    # Remove numbers
    df.loc[:, 'document'] = df.loc[:, 'document'].str.replace(r'\d+', '')
    
    # Remove Python syntax
    df.loc[:, 'document'] = df.loc[:, 'document'].apply(remove_python_syntax)
    
    # Remove URLs
    df.loc[:, 'document'] = df.loc[:, 'document'].apply(lambda x: re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', x))

    # Remove images
    df.loc[:, 'document'] = df.loc[:, 'document'].apply(lambda x: re.sub(r'\bimage\S+', '', x))

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    df.loc[:, 'document'] = df.loc[:, 'document'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    df.loc[:, 'document'] = df.loc[:, 'document'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
    
#     # Remove infrequent words if enabled
#     if remove_infrequent:
#         df['document'] = df['document'].apply(lambda x: remove_infrequent_words(x, infrequent_threshold))

    # Stemming
    stemmer = PorterStemmer()
    df.loc[:, 'document'] = df.loc[:, 'document'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))
    
    # Remove single characters
    df['document'] = df['document'].apply(remove_single_characters)
    
    # Remove single-character words
    df['document'] = df['document'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 1]))
    
    # Remove punctuation
    df['document'] = df['document'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
    
    # Remove numbers or words starting or ending with a number
#     df['document'] = df['document'].apply(lambda x: re.sub(r'\b\d+\w*\b|\b\w*\d+\b', '', x))
    
    # Remove non-word and non-space characters, words containing digits, and emojis
    df['document'] = df['document'].apply(lambda x: re.sub(r'[^\w\s]|(\w*\d\w*)|[\U0001F300-\U0001F6FF]', '', x))


    return df

def remove_python_syntax(text):
    # Remove Python code blocks enclosed in triple backticks
    text = re.sub(r'```[\s\S]*?```', '', str(text))

    # Remove Python code blocks enclosed in double backticks
    text = re.sub(r'``[\s\S]*?``', '', str(text))

    # Remove Python code blocks enclosed in single backticks
    text = re.sub(r'`[\s\S]*?`', '', str(text))
    
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', str(text))
    
    return text

def remove_infrequent_words(text, threshold):
        words = text.split()
        word_counts = Counter(words)
        infrequent_words = [word for word, count in word_counts.items() if count < threshold]
        return ' '.join([word for word in words if word not in infrequent_words])
    
def remove_single_characters(text):
        return ' '.join([word for word in text.split() if len(word) > 1])

In [ ]:
process_df = preprocess_text(filtered_df)
process_df

In [ ]:
with pd.option_context('display.max_colwidth', None):  
    display(process_df["document"][32342])

In [ ]:
process_df["body"][32342]

In [ ]:
def build_unigram_bigram(texts):
    # Tokenize the texts
    tokenized_texts = [word_tokenize(text) for text in texts]

    # Build bigrams
    bigram = Phrases(tokenized_texts, min_count=5, threshold=10)
    bigram_phraser = Phraser(bigram)
    bigram_texts = [bigram_phraser[doc] for doc in tokenized_texts]

    return tokenized_texts, bigram_texts

In [ ]:
def find_optimal_num_topics(texts, num_topics_range):
    
    tokenized_texts, bigram_texts = build_unigram_bigram(texts)
#     print(tokenized_texts)
    # Tokenize the texts
#     tokenized_texts = [text.split() for text in texts]

    # Create the dictionary
    dictionary = Dictionary(bigram_texts)

    # Create the corpus
    corpus = [dictionary.doc2bow(text) for text in bigram_texts]

    coherence_scores = []
    for num_topics in num_topics_range:
        # Build the LDA model
        lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, iterations=11, random_state=100, passes=10)

        # Compute coherence score
        coherence_model = CoherenceModel(model=lda_model, texts=bigram_texts, dictionary=dictionary, coherence='c_v')
        coherence_score = coherence_model.get_coherence()
        coherence_scores.append(coherence_score)

    return coherence_scores

In [ ]:
num_topics_range = [i for i in range(5, 60, 1)]  # Adjust the range as needed

In [ ]:
coherence_scores = find_optimal_num_topics(process_df['document'].to_list(), num_topics_range)

In [ ]:
def plot_coherence_scores(num_topics_range, coherence_scores):
    # Plot the coherence scores
    plt.figure(figsize=(20,10))
    plt.plot(num_topics_range, coherence_scores, marker='o')
    plt.xlabel('Number of Topics')
    plt.ylabel('Coherence Score')
    plt.title('Coherence Scores vs. Number of Topics')
    plt.xticks(num_topics_range)
    plt.show()

In [ ]:
# Call the function to plot the coherence scores
plot_coherence_scores(num_topics_range, coherence_scores)

In [ ]:
def train_lda_model(texts, num_topics=6):
    tokenized_texts, bigram_texts = build_unigram_bigram(texts)

    # Create the dictionary
    dictionary = Dictionary(bigram_texts)

    # Create the corpus
    corpus = [dictionary.doc2bow(text) for text in bigram_texts]

    # Build the LDA model
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, iterations=11, random_state=100, passes=10)

    # Get the topics and keywords
    topics = lda_model.show_topics(num_topics=num_topics, formatted=False)

    # Extract the keywords from the topics
    topic_keywords = []
    for topic_id, topic in topics:
        keywords = [word for word, _ in topic]
        topic_keywords.append(keywords)

    return lda_model, topics, dictionary, corpus

In [ ]:
model_, topics_, dictionnary_, corpus_ = train_lda_model(process_df['document'].to_list())

In [ ]:
def visualize_topics(lda_model, corpus, dictionary):
    pyLDAvis.enable_notebook()
    # Convert the Gensim LDA model to pyLDAvis format
    vis_data = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

    # Display the interactive visualization
    return pyLDAvis.display(vis_data)

In [ ]:
visualize_topics(model_, corpus_, dictionnary_)

In [ ]:
model_.print_topics()

In [ ]:
filtered_df